In [59]:
import os
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import PatternFill
from openpyxl.utils.dataframe import dataframe_to_rows

# 폴더 경로
path=input("취합 할 파일이 있는 폴더 경로를 입력하세요")

# 결과를 저장할 엑셀 파일 경로
output_file = input(".xlsx를 포함한 저장할 파일의 제목을 입력하세요")
output_path = input("출력 파일을 저장할 폴더 경로를 입력하세요")

# 폴더 내의 모든 CSV 파일 목록 가져오기
csv_files = []
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('.xlsx'):
            csv_files.append(os.path.join(root, file))

# 결과를 저장할 리스트 초기화
combined_data=pd.DataFrame(columns=["날짜","계측기 정보","정상 데이터 비율"])

# 각 CSV 파일에 대한 결과 계산
for file in csv_files:
    # CSV 파일을 DataFrame으로 불러오기
    df = pd.read_excel(file)
    new_columns = df['파일명'].str.split('[/\\\\-_#.]', expand=True)
    new_columns['계측기 정보'] = new_columns.iloc[:,-6] + '_' + new_columns.iloc[:,-4] + '_' + new_columns.iloc[:,-3]
    columns_to_keep = ["계측기 정보"]
    new_columns = new_columns[columns_to_keep]
    df=pd.concat([new_columns,df],axis=1)
    columns_to_keep = ["날짜","계측기 정보","정상 데이터 비율"]
    df = df[columns_to_keep]
    combined_data = pd.concat([combined_data, df], ignore_index=True)
df_pivot = combined_data.pivot_table(index='계측기 정보', columns='날짜', values='정상 데이터 비율',aggfunc="sum")

df_pivot = df_pivot.apply(lambda x: x.str.rstrip('%').astype(float) / 100)

df_pivot['데이터 안정화(3일 연속 99.7%)'] = df_pivot.iloc[:, :].apply(lambda row: 'O' if (row.astype(float) >= 0.997).rolling(window=3, min_periods=1).sum().eq(3).any() else 'X', axis=1)
df_pivot["데이터 안정화(마지막 일 기준 3일 연속 99.7%)"] = df_pivot.iloc[:, -2:-5:-1].apply(lambda row: 'O' if (row.astype(float) >= 0.997).all() else 'X', axis=1)
df_pivot.iloc[:,:-2]=(df_pivot.iloc[:,:-2]*100).round(2)


output_file_path = os.path.join(output_path, output_file)


def highlight_cells(df):
    # 엑셀 파일 생성
    wb = Workbook()
    ws = wb.active

    # 데이터 프레임을 엑셀에 입력
    for r in dataframe_to_rows(df, index=True, header=True):
        ws.append(r)


    # 조건에 따라 셀 색상 변경
    for row in ws.iter_rows(min_row=2, min_col=1, max_col=len(df.columns)):
        for cell in row:
            try:
                if cell.value is not None and float(cell.value) < 99.7:
                    cell.fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
            except ValueError:
                pass

#     # 엑셀 파일 저장
#     wb.save('example.xlsx')
    wb.save(output_file_path)


# # 데이터 프레임을 엑셀 파일로 저장하고 조건에 따라 셀 색상 변경
highlight_cells(df_pivot)

print("결과가", output_file_path, "파일에 저장되었습니다.")


결과가 Z:\ESNJ  계측기 로그\230816 결과파일 취합 테스트.xlsx 파일에 저장되었습니다.
